In [ ]:
districts = [
    "amaravati",
    "anantapur",
    "shri sathya sai",
    "chittoor",
    "tirupati",
    "east godavari",
    "kakinada",
    "dr. br ambedkar konaseema",
    "guntur",
    "bapatla",
#     "badass",
    "palnadu",
    "krishna",
    "ntr",
    "kurnool",
    "nandyala",
    "brightness",
    "sripotti sriramulu nellore",
    "srikakulam",
    "anakapalli",
    "visakhapatnam",
    "alluri sitaramaraj",
    "vizianagaram",
    "parvathipuram manyam",
    "west godavari",
    "elur",
    "ysr",
#     "brother",
    "annamayya"
]

In [ ]:
import threading
import time
import Sakshi_Main as sa
import eenadu_Main as ee

def process_district_news(news_source, district):
    max_retries = 3
    for _ in range(max_retries):
        try:
            news_source.TodaysNews(district)
            print(f"Processing news from {news_source.__name__} for district {district} completed.")
            time.sleep(2)
            break  
        except Exception as e:
            print(f"Error processing news from {news_source.__name__} for district {district}: {e}")
            time.sleep(2)  
    else:
        print(f"Failed to process news from {news_source.__name__} for district {district}")

def process_news_for_source(news_source, districts):
    threads = []
    for district in districts:
        thread = threading.Thread(target=process_district_news, args=(news_source, district))
        threads.append(thread)

    
    for thread in threads:
        thread.start()

   
    for thread in threads:
        thread.join()



eenadu_thread = threading.Thread(target=process_news_for_source, args=(ee, districts))
eenadu_thread.start()
eenadu_thread.join()

print("All news processing is complete.")


In [ ]:
df_news = pd.read_csv('Data/Sakshi/amaravati/Sakshi_2023-08-18_data.csv')
df_news

In [ ]:
Trans_list = list(df_news.News)

In [ ]:
Trans_list

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-3.3B")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-3.3B")

In [ ]:
Trans_text = []

for n in Trans_list:
    article = f"translate Te: {n} to En:"
    inputs = tokenizer(article, return_tensors="pt", padding=True, truncation=True)
    translated_tokens = model.generate(**inputs)
    translated_text = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    Trans_text.append(translated_text)


In [ ]:
Trans_text = []
for n in Trans_list:
    article = n
    inputs = tokenizer(article, return_tensors="pt", source_lang="te", target_lang="en")
    translated_tokens = model.generate(**inputs)
    Trans_text.append(tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0])

In [ ]:
Trans_text

In [ ]:
from textblob import TextBlob
import sys
#import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import os
import nltk
#import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list['text'].items():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        tw_list.loc[index, 'sentiment'] = 'negative'
    elif pos > neg:
        tw_list.loc[index, 'sentiment'] = 'positive'
    else:
        tw_list.loc[index, 'sentiment'] = 'neutral'
        tw_list.loc[index, 'neg'] = neg
        tw_list.loc[index, 'neu'] = neu
        tw_list.loc[index, 'pos'] = pos
        tw_list.loc[index, 'compound'] = comp
tw_list

In [ ]:
tw_list_negative = tw_list[tw_list['sentiment']=='negative']
tw_list_positive = tw_list[tw_list['sentiment']=='positive']
tw_list_neutral = tw_list[tw_list['sentiment']=='neutral']

In [ ]:
def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])
#Count_values for sentiment
sentiment_df = count_values_in_column(tw_list,'sentiment')

In [ ]:
sentiment_df.reset_index()

In [ ]:
import plotly.express as px
fig = px.pie(sentiment_df,values=sentiment_df.Percentage,names=sentiment_df.index)
fig.show()

In [ ]:
#Calculating tweet’s lenght and word count
tw_list["text_len"] = tw_list['text'].astype(str).apply(len)
tw_list["text_word_count"] = tw_list['text'].apply(lambda x: len(str(x).split()))
round(pd.DataFrame(tw_list.groupby("sentiment").text_len.mean()),2)


In [ ]:
round(pd.DataFrame(tw_list.groupby('sentiment').text_word_count.mean()),2)

In [ ]:
#Removing Punctuation
def remove_punct(text):
    text = ''.join([char for char in text if char not in string.punctuation])
    text = re.sub('[0–9]+', '', text)
    return text
tw_list['punct'] = tw_list['text'].apply(lambda x: remove_punct(x))

#Appliyng tokenization
def tokenization(text):
    text = re.split('\W+', text)
    return text
tw_list['tokenized'] = tw_list['punct'].apply(lambda x: tokenization(x.lower()))

#Removing stopwords
stopword = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    text = [word for word in text if word not in stopword]
    return text

tw_list['nonstop'] = tw_list['tokenized'].apply(lambda x: remove_stopwords(x))

#Appliyng Stemmer
ps = nltk.PorterStemmer()
def stemming(text):
    text = [ps.stem(word) for word in text]
    return text
tw_list['stemmed'] = tw_list['nonstop'].apply(lambda x: stemming(x))

#Cleaning Text
def clean_text(text):
    text_lc = "".join([word.lower() for word in text if word not in string.punctuation]) # remove puntuation
    text_rc = re.sub('[0-9]+', '', text_lc)
    tokens = re.split('\W+', text_rc)    # tokenization
    text = [ps.stem(word) for word in tokens if word not in stopword]  # remove stopwords and stemming
    return text
tw_list.head()

In [ ]:
#Appliyng Countvectorizer
countVectorizer = CountVectorizer(analyzer=clean_text) 
countVector = countVectorizer.fit_transform(tw_list['text'])
print('{} Number of reviews has {} words'.format(countVector.shape[0], countVector.shape[1]))

In [ ]:
count_vect_df = pd.DataFrame(countVector.toarray(), columns=countVectorizer.get_feature_names_out())
count_vect_df.head()

In [ ]:
count = pd.DataFrame(count_vect_df.sum())
countdf = count.sort_values(0,ascending=False).head(20)
countdf[:11]

In [ ]:
#Function to ngram
def get_top_n_gram(corpus,ngram_range,n=None):
    vec = CountVectorizer(ngram_range=ngram_range,stop_words = 'english').fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]
#n2_bigram
n2_bigrams = get_top_n_gram(tw_list['text'],(2,2),10)
n2_bigrams

In [ ]:
n3_trigrams = get_top_n_gram(tw_list['text'],(3,3),20)
n3_trigrams

In [ ]:
n4_trigrams = get_top_n_gram(tw_list['text'],(4,4),10)
n4_trigrams

In [ ]:
n5_trigrams = get_top_n_gram(tw_list['text'],(5,5),10)
n5_trigrams

In [ ]:
n6_trigrams = get_top_n_gram(tw_list['text'],(6,6),10)
n6_trigrams

In [ ]:
word_list = [i[0] for i in n3_trigrams]

In [ ]:
comment_words = ''
stopwords = set(STOPWORDS)

# iterate through the csv file
for val in word_list:
    val = str(val)
    tokens = val.split()
    for i in range(len(tokens)):
        tokens[i] = tokens[i].lower()
    comment_words += " ".join(tokens)+" "

wordcloud = WordCloud(width = 800, height = 800,
background_color ='white',
stopwords = stopwords,
min_font_size = 10).generate(comment_words)

# plot the WordCloud image					
plt.figure(figsize = (8, 8), facecolor = None)
plt.imshow(wordcloud)
plt.axis("off")
plt.tight_layout(pad = 0)

plt.show()

# Summary

In [ ]:
top_10 = pd.DataFrame()
top_10['text'] = df1.Trans_text[:10]
top_10 = top_10.dropna()

In [ ]:
remove_rt = lambda x:re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub(re.escape("(@[A-Za-z0–9]+)|([⁰-9A-Za-z \t])|(\w+:\/\/\S+)")," ",x)
top_10["text"] = top_10.text.map(remove_rt).map(rt)
top_10["text"] = top_10.text.str.lower()
top_10.head(10)

In [ ]:
Text = ''
for i in list(top_10.text):
    Text+=i
Text

In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

result = summarizer(Text, max_length=200, min_length=100, do_sample=False)
result = result[0]['summary_text']

In [ ]:
result